In [1]:
import requests
from bs4 import BeautifulSoup
from urllib.request import urlopen,Request
from selenium import webdriver
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
import pandas as pd
import time

In [2]:
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/108.0.0.0 Safari/537.36'
}

In [3]:
airbnb_base = 'https://www.airbnb.com'

In [4]:
airbnb_url = 'https://www.airbnb.com/s/Broga-Hill-Park--Jalan-Broga--Semenyih--Selangor--Malaysia/homes?tab_id=home_tab&refinement_paths%5B%5D=%2Fhomes&flexible_trip_lengths%5B%5D=one_week&price_filter_input_type=0&price_filter_num_nights=5&query=Broga%20Hill%20Park%2C%20Jalan%20Broga%2C%20Semenyih%2C%20Selangor&place_id=ChIJQwDTU-_RzTERgPctqB8XY-8&date_picker_type=calendar&source=structured_search_input_header&search_type=autocomplete_click'

In [5]:
response = requests.get(airbnb_url)
response

<Response [200]>

In [7]:
airbnb_list = []
next_url = airbnb_url
while next_url:
    soup = BeautifulSoup(requests.get(next_url).content, 'html.parser')

    listings = soup.find_all('div', class_='c4mnd7m dir dir-ltr')

    for listing in listings:
        price = listing.find('span', class_='a8jt5op dir dir-ltr').text
        title = listing.find('span', class_='t6mzqp7 dir dir-ltr').text
        name = listing.find('div', class_='t1jojoys dir dir-ltr').text
        rating = listing.find('span', class_='r1dxllyb dir dir-ltr')
        try:
            superhost = listing.find('div', class_='t1mwk1n0 dir dir-ltr').text
        except:
            superhost = 'n/a'
        listing_info = {
            'price': price,
            'title': title,
            'name': name,
            'superhost': superhost,
            'rating': rating
            
        }
        airbnb_list.append(listing_info)

        # Check for the link to the detailed page
        detailed_links = listing.find_all('a', class_='bn2bl2p dir dir-ltr')
        if detailed_links:
            for link in detailed_links:
                detailed_url = airbnb_base + link['href']
#                 driver = webdriver.Chrome()
#                 driver.get(detailed_url)
#                 page_detailed = driver.page_source
#                 driver.quit()
#                 detail_soup = BeautifulSoup(page_detailed)
                
                driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
                driver.get(detailed_url)

                driver.maximize_window()
                time.sleep(5)
                soup2=BeautifulSoup(driver.page_source, 'lxml')
                try:
                    review = soup2.find('span', class_='_s65ijh7').text
                except:
                    review = 'n/a'
                try:
                    rooms_info = soup2.find('ol', class_='lgx66tx dir dir-ltr').text
                except:
                    rooms_info = 'n/a'

#                 results = detail_soup.find('div', class_='_1byskwn')
#                 offers = results.find_all('div', class_='_19xnuo97')
#                 for i, offer in enumerate(offers):
#                     listing_info[f'offer_{i+1}'] = offer.text

                airbnb_list.append({**listing_info, 'review': review, 'rooms_info': rooms_info,'link': detailed_url})

    # Check for a link to the next page
    next_button = soup.find('a', class_='_1bfat5l')
    if next_button:
        # Update the URL to scrape the next page
        next_url = airbnb_base + next_button['href']
    else:
        # If no link is found, set the URL to None to exit the loop
        next_url = None


In [8]:
airbnb_list

[{'price': 'RM1,927 per night',
  'title': 'TTS Lake Villa @ Broga, Semenyih',
  'name': 'Villa in Semenyih',
  'superhost': 'Superhost',
  'rating': <span aria-hidden="true" class="r1dxllyb dir dir-ltr">4.86 (96)</span>},
 {'price': 'RM1,927 per night',
  'title': 'TTS Lake Villa @ Broga, Semenyih',
  'name': 'Villa in Semenyih',
  'superhost': 'Superhost',
  'rating': <span aria-hidden="true" class="r1dxllyb dir dir-ltr">4.86 (96)</span>,
  'review': '96 reviews',
  'rooms_info': '16+ guests ·  · 4 bedrooms ·  · 15 beds ·  · 3 baths',
  'link': 'https://www.airbnb.com/rooms/52588088?adults=1&children=0&infants=0&pets=0&check_in=2023-02-27&check_out=2023-03-04&previous_page_section_name=1000'},
 {'price': 'RM3,138 per night',
  'title': 'TTS Beach Village @ Broga, Semenyih',
  'name': 'Villa in Semenyih',
  'superhost': 'Superhost',
  'rating': <span aria-hidden="true" class="r1dxllyb dir dir-ltr">5.0 (15)</span>},
 {'price': 'RM3,138 per night',
  'title': 'TTS Beach Village @ Broga,

In [9]:
df = pd.DataFrame(airbnb_list)
df

,price,title,name,superhost,rating,review,rooms_info,link
0,"RM1,927 per night","TTS Lake Villa @ Broga, Semenyih",Villa in Semenyih,Superhost,[4.86 (96)],NaN,NaN,NaN
1,"RM1,927 per night","TTS Lake Villa @ Broga, Semenyih",Villa in Semenyih,Superhost,[4.86 (96)],96 reviews,16+ guests · · 4 bedrooms · · 15 beds · · 3...,https://www.airbnb.com/rooms/52588088?adults=1...
2,"RM3,138 per night","TTS Beach Village @ Broga, Semenyih",Villa in Semenyih,Superhost,[5.0 (15)],NaN,NaN,NaN
3,"RM3,138 per night","TTS Beach Village @ Broga, Semenyih",Villa in Semenyih,Superhost,[5.0 (15)],15 reviews,16+ guests · · 5 bedrooms · · 22 beds · · 6...,https://www.airbnb.com/rooms/73086887394461725...
4,"RM1,716 per night",Mountain view villa with entertainment room,Villa in Semenyih,n/a,[4.6 (15)],NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
535,RM470 per night,Tokido Executive Water Villa Private Unit,Chalet in Port Dickson,n/a,[4.74 (88)],88 reviews,4 guests · · 1 bedroom · · 2 beds · · 1 bath,https://www.airbnb.com/rooms/37679839?adults=1...
536,RM144 per night,Family Inclined 2bedroom Suite @ Arte Plus Ampang,Apartment in Kuala Lumpur,n/a,[New],NaN,NaN,NaN
537,RM144 per night,Family Inclined 2bedroom Suite @ Arte Plus Ampang,Apartment in Kuala Lumpur,n/a,[New],n/a,10 guests · · 2 bedrooms · · 7 beds · · 2 b...,https://www.airbnb.com/rooms/80258763560683701...
538,RM230 per night,Cozy Little Muji Home Kepong Nice View MRT E03A D,Condo in Kuala Lumpur,Superhost,[New],NaN,NaN,NaN


In [10]:
#convert to csv
df.to_csv('Airbnbsemnyih.csv')